In [1]:
from autompc.sysid.autoselect import AutoSelectModel
from autompc.tuning.model_evaluator import CrossValidationModelEvaluator, HoldoutModelEvaluator, ModelEvaluator
from autompc.model_metalearning.meta_utils import load_data, load_cfg

Loading AutoMPC...


/home/baoyu/anaconda3/envs/autompc/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Finished loading AutoMPC


In [2]:
from distutils.log import info
import os, glob
import pickle
import json
from collections import namedtuple
import numpy as np
import pandas as pd

In [3]:
data_path = '/home/baoyu/baoyul2/autompc/autompc/model_metalearning/meta_data'
cfg_path = '/home/baoyu/baoyul2/autompc/autompc/model_metalearning/meta_cfg'

In [4]:
eval_metric="rmse"
eval_horizon=1
eval_quantile=None
eval_folds=3

In [5]:
names = ["HalfCheetah-v2", "HalfCheetahSmall-v2"]
output_results_dictionary = {}

# data
for data_name in names:
    print(data_name)
    system, trajs = load_data(data_path, data_name)
    scores = []
    # config
    for cfg_name in names:
        cfg = load_cfg(cfg_path, cfg_name)
        print(cfg)
        model = AutoSelectModel(system)
        model.set_config(cfg)
        evaluator = CrossValidationModelEvaluator(trajs, eval_metric, horizon=eval_horizon, quantile=eval_quantile, num_folds=eval_folds,
                    rng=np.random.default_rng(100))
        score = evaluator(model)
        scores.append(score)
    output_results_dictionary[data_name] = scores

HalfCheetah-v2
Configuration(values={
  'MLP:batchnorm': False,
  'MLP:hidden_size_1': 128,
  'MLP:hidden_size_2': 128,
  'MLP:lr': 0.001,
  'MLP:n_hidden_layers': '2',
  'MLP:nonlintype': 'relu',
  'model': 'MLP',
})

use_cuda=True
torch.cuda.is_available()=True
self._device=device(type='cuda')
Cuda is used for GPytorch
self.net.parameters().__next__().device=device(type='cuda', index=0)
100%|██████████| 200/200 [00:45<00:00,  4.39it/s]
self.net.parameters().__next__().device=device(type='cuda', index=0)
100%|██████████| 200/200 [00:44<00:00,  4.48it/s]
self.net.parameters().__next__().device=device(type='cuda', index=0)
100%|██████████| 200/200 [00:44<00:00,  4.51it/s]
Configuration(values={
  'Koopman:lasso_alpha': 0.006765721868566048,
  'Koopman:method': 'lasso',
  'Koopman:poly_basis': 'false',
  'Koopman:trig_basis': 'false',
  'model': 'Koopman',
})

use_cuda=True
torch.cuda.is_available()=True
self._device=device(type='cuda')
Cuda is used for GPytorch
Call Lasso (1000 iters)
C

In [6]:
output_results_dictionary

{'HalfCheetah-v2': [2.6049768303569194, 4.556939720609314],
 'HalfCheetahSmall-v2': [6.563847631056436, 4.014697565628448]}

In [8]:
matrix = pd.DataFrame(data=output_results_dictionary)
matrix = matrix.transpose()
matrix

,0,1
HalfCheetah-v2,2.604977,4.556940
HalfCheetahSmall-v2,6.563848,4.014698


In [9]:
matrix_path = '/home/baoyu/baoyul2/autompc/autompc/model_metalearning/meta_matrix'
output_file_name = os.path.join(matrix_path, 'matrix.pkl')
print("Dumping to ", output_file_name)
with open(output_file_name, 'wb') as fh:
    pickle.dump(matrix, fh)

Dumping to  /home/baoyu/baoyul2/autompc/autompc/model_metalearning/meta_matrix/matrix.pkl
